In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

In [2]:
from typing import Dict
from acctrack.io.athena_raw_root import AthenaRawRootReader
from acctrack.io import utils_athena_raw_root as utils_raw_root

import awkward as ak
import numpy as np
import uproot

In [11]:
input_dir = "/media/DataOcean/data/ITk/BugFixedSamples"
filename = input_dir + "/GNN4Itk__mc15_14TeV.600012.PhPy8EG_A14_ttbar_hdamp258p75_nonallhad.recon.RDO.e8185_s3770_s3773_r14429__J001.root"

In [37]:
reader = AthenaRawRootReader(input_dir)

/media/DataOcean/data/ITk/BugFixedSamples contains  1 files and total 200 events.


In [38]:
reader.read_file()

['PIXEL' 'STRIP']
[-2  0  2]


<TTree 'GNN4ITk' (96 branches) at 0x7f67d841fc70>

In [21]:
reader.read()

In [29]:
reader.spacepoints

,hit_id,x,y,z,cluster_index_1,cluster_index_2
0,0,-33.875000,6.775000,-265.000,0,-1
1,1,-33.925000,-7.550000,-265.000,1,-1
2,2,-34.000000,0.425000,-265.000,2,-1
3,3,-34.050000,-3.925000,-265.000,3,-1
4,4,-34.125000,0.100000,-265.000,4,-1
...,...,...,...,...,...,...
266665,266665,901.356972,-106.466514,2854.251,387312,387329
266666,266666,896.432888,-142.083029,2854.251,387314,387326
266667,266667,927.103764,-170.343086,2854.251,387316,387324
266668,266668,919.038052,-168.698163,2854.251,387316,387325


In [34]:
reader.clusters

,cluster_id,cluster_x,cluster_y,cluster_z,barrel_endcap,layer_disk,eta_module,phi_module,side,pixel_count,...,glob_phi,eta_angle,phi_angle,norm_x,norm_y,norm_z,hardware,subevent,barcode,particle_id
0,1,-33.875000,6.775000,-265.000,-2,0,0,0,0,1,...,-2.356194,1.249046,1.249046,0.0,0.0,-1.0,PIXEL,3270.0,564.0,32700000564
1,2,-33.925000,-7.550000,-265.000,-2,0,0,0,0,4,...,-2.158799,0.982794,0.785398,0.0,0.0,-1.0,PIXEL,NaN,NaN,None
2,3,-34.000000,0.425000,-265.000,-2,0,0,0,0,2,...,-2.034444,1.249046,0.982794,0.0,0.0,-1.0,PIXEL,3119.0,1209.0,31190001209
3,4,-34.050000,-3.925000,-265.000,-2,0,0,0,0,2,...,-2.034444,1.249046,0.982794,0.0,0.0,-1.0,PIXEL,3253.0,155.0,32530000155
4,5,-34.125000,0.100000,-265.000,-2,0,0,0,0,2,...,-2.677945,0.982794,1.249046,0.0,0.0,-1.0,PIXEL,3107.0,306.0,31070000306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389017,387328,930.538524,-115.703549,2860.749,2,5,5,63,1,1,...,1.419725,1.324637,0.006379,0.0,0.0,-1.0,STRIP,NaN,NaN,None
389018,387329,930.558368,-115.543862,2860.749,2,5,5,63,1,1,...,1.419725,1.324637,0.006379,0.0,0.0,-1.0,STRIP,NaN,NaN,None
389019,387330,931.362416,-108.873495,2860.749,2,5,5,63,1,2,...,1.418207,1.115861,0.006380,0.0,0.0,-1.0,STRIP,NaN,NaN,None
389020,387331,931.775955,-105.275959,2860.749,2,5,5,63,1,2,...,1.415171,0.803246,0.006380,0.0,0.0,-1.0,STRIP,NaN,NaN,None


In [33]:
reader.truth.loc[0]

hit_id                  0
x                 -33.875
y                   6.775
z                  -265.0
cluster_index_1         0
cluster_index_2        -1
particle_id           nan
cluster_x_1          -1.0
cluster_y_1          -1.0
cluster_z_1          -1.0
barrel_endcap        -1.0
layer_disk           -1.0
eta_module           -1.0
phi_module           -1.0
pixel_count          -1.0
charge_count         -1.0
loc_eta              -1.0
loc_phi              -1.0
loc_direction1       -1.0
loc_direction2       -1.0
loc_direction3       -1.0
glob_eta             -1.0
glob_phi             -1.0
eta_angle            -1.0
phi_angle            -1.0
norm_x               -1.0
norm_y               -1.0
norm_z               -1.0
hardware               -1
subevent             -1.0
barcode              -1.0
cluster_x_2          -1.0
cluster_y_2          -1.0
cluster_z_2          -1.0
Name: 0, dtype: object

In [10]:
np.unique(reader.clusters["barrel_endcap"]), np.unique(reader.clusters["hardware"])

(array([-2,  0,  2], dtype=int32), array(['PIXEL', 'STRIP'], dtype=object))

In [15]:
region_labels = dict([(1, {"hardware": "PIXEL", "barrel_endcap": -2}),
                        (2, {"hardware": "STRIP", "barrel_endcap": -2}),
                        (3, {"hardware": "PIXEL", "barrel_endcap": 0}),
                        (4, {"hardware": "STRIP", "barrel_endcap": 0}),
                        (5, {"hardware": "PIXEL", "barrel_endcap": 2}),
                        (6, {"hardware": "STRIP", "barrel_endcap": 2})
                        ])

In [16]:
region_labels

{1: {'hardware': 'PIXEL', 'barrel_endcap': -2},
 2: {'hardware': 'STRIP', 'barrel_endcap': -2},
 3: {'hardware': 'PIXEL', 'barrel_endcap': 0},
 4: {'hardware': 'STRIP', 'barrel_endcap': 0},
 5: {'hardware': 'PIXEL', 'barrel_endcap': 2},
 6: {'hardware': 'STRIP', 'barrel_endcap': 2}}

In [20]:
tree = reader.read_file()

TypeError: '<' not supported between instances of 'str' and 'int'

In [26]:
for batch in tree.iterate(step_size=1, filter_name="SP*", library="np"):
    print(batch)
    break

{'SPindex': array([array([     0,      1,      2, ..., 266667, 266668, 266669], dtype=int32)],
      dtype=object), 'SPx': array([array([-33.875     , -33.925     , -34.        , ..., 927.10376355,
              919.03805227, 911.97417735])                                ],
      dtype=object), 'SPy': array([array([   6.775     ,   -7.55      ,    0.425     , ..., -170.34308606,
              -168.69816319, -172.37874461])                                  ],
      dtype=object), 'SPz': array([array([-265.   , -265.   , -265.   , ..., 2854.251, 2854.251, 2854.251])],
      dtype=object), 'SPCL1_index': array([array([     0,      1,      2, ..., 387316, 387316, 387317], dtype=int32)],
      dtype=object), 'SPCL2_index': array([array([    -1,     -1,     -1, ..., 387324, 387325, 387323], dtype=int32)],
      dtype=object)}


In [16]:
len(batch["CLhardware"][0].tolist())

387332

In [21]:
for batch in tree.iterate(step_size=1, filter_name=utils_raw_root.cluster_link_branch_names+['CLindex'], library="np"):
    print(batch)
    break

{'CLindex': array([array([     1,      2,      3, ..., 387330, 387331, 387332], dtype=int32)],
      dtype=object), 'CLparticleLink_eventIndex': array([<STLVector [[3270], [], [3119], [3253], ..., [], [], [], []] at 0x7fcc0b04b6a0>],
      dtype=object), 'CLparticleLink_barcode': array([<STLVector [[564], [], [1209], [155], [306], ..., [], [], [], []] at 0x7fca5d8aa8b0>],
      dtype=object)}


In [23]:
subevent_name, barcode_name = utils_raw_root.cluster_link_branch_names

In [25]:
batch[subevent_name][0].tolist()

[[3270],
 [],
 [3119],
 [3253],
 [3107],
 [],
 [3194, 3406],
 [],
 [],
 [],
 [3233],
 [3205],
 [3218],
 [3218],
 [],
 [],
 [3088],
 [3182],
 [],
 [3269],
 [3267],
 [],
 [3240],
 [3251],
 [3146],
 [3129],
 [3085],
 [3249],
 [],
 [],
 [3140],
 [],
 [],
 [],
 [3261],
 [3176],
 [3234],
 [3158],
 [3168],
 [3194],
 [3121],
 [3148],
 [],
 [3119],
 [3176],
 [],
 [3267],
 [],
 [3217],
 [3129],
 [0],
 [3124],
 [3229],
 [3220],
 [],
 [3229],
 [3214],
 [],
 [],
 [3182],
 [],
 [3105],
 [3194],
 [],
 [3111],
 [],
 [],
 [3253],
 [],
 [3171],
 [3194],
 [3270],
 [],
 [3119],
 [],
 [3218],
 [3218],
 [],
 [3107],
 [],
 [3233],
 [],
 [3182],
 [3088],
 [],
 [3269],
 [],
 [3267],
 [3240],
 [3146],
 [3129],
 [],
 [3085],
 [],
 [3269],
 [],
 [3140],
 [3158],
 [],
 [3168],
 [3194],
 [3233],
 [3202],
 [3229],
 [3206],
 [],
 [3176],
 [],
 [3176],
 [3244],
 [],
 [],
 [],
 [3229],
 [0],
 [3124],
 [3182],
 [],
 [3229],
 [3214],
 [3105],
 [3194],
 [],
 [],
 [3253],
 [3111],
 [],
 [],
 [],
 [3270],
 [3119],
 [],
 [32

In [ ]:
for clusters in tree.iterate(step_size=1, filter_name=utils.cluster_branch_names+[''], library="np"):
    print(clusters)
    break

In [ ]:
arrays

In [ ]:
particle_branch_prefix = "Part_"
# "vParentID", "vParentBarcode" are not used.
particle_columns = [
    "event_number", "barcode", 
    "px", "py", "pz", "pt", "eta", 
    "vx", "vy", "vz",
    "radius", "status", "charge", "pdg_id",
    "passed", "vProdNin", "vProdNout", "vProdStatus", 
    "vProdBarcode"
]

particle_branch_names = [particle_branch_prefix + col for col in particle_columns]

In [ ]:
particles = tree.arrays(particle_branch_names, library="np")

In [ ]:
type(particles)

In [ ]:
particles["Part_event_number"][0]

In [ ]:
def create_arrays(branch_arrays: Dict[str, np.ndarray]):
    variables = list(branch_arrays.keys())
    num_evts = len(branch_arrays[variables[0]])
    all_arrays = []
    print("num_evts:", num_evts)
    for i in range(num_evts):
        inputs = [branch_arrays[x][i] for x in variables]
        array = np.stack(inputs, axis=1)
        all_arrays.append(array)

    return all_arrays

In [ ]:
len(particles[list(particles.keys())[0]])

In [ ]:
par_array = create_arrays(particles, 0)

In [ ]:
par_array.shape

In [ ]:
particle_pt = particles[particle_branch_prefix + "pt"]

In [ ]:
type(particle_pt), len(particle_pt[0])

In [ ]:
df = ak.to_dataframe(particle_pt[0])

In [ ]:
df

In [ ]:
df.loc[(0, 0)]